In [96]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from config import DB_USERNAME, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# import warnings
# warnings.filterwarnings("ignore")

In [97]:
# Create a connection string with the postgreSQL info
database_access = f"postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"


In [98]:
# Establish a connection to the PostgreSQL database 
engine = create_engine(database_access)


In [99]:
# Write a SQL query to select data from the existing view in PostgreSQL 
query = 'SELECT * FROM "Wines"'


In [100]:
# Use Pandas to read the data from the SQL query and convert into a DataFrame 
wines_df = pd.read_sql_query(query, engine)


In [101]:
# Display the Dataframe
wines_df.head()

,id,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol_wine,quality,quality_categorisation
0,4799,White,6.2,0.260,0.20,8.00,0.047,35.0,111.0,0.99445,3.11,0.42,10.4,6,1
1,1159,Red,10.2,0.410,0.43,2.20,0.110,11.0,37.0,0.99728,3.16,0.67,10.8,5,0
2,315,Red,7.1,0.350,0.29,2.50,0.096,20.0,53.0,0.99620,3.42,0.65,11.0,6,1
3,31,Red,6.9,0.685,0.00,2.50,0.105,22.0,37.0,0.99660,3.46,0.57,10.6,6,1
4,3010,White,6.7,0.250,0.31,1.35,0.061,30.5,218.0,0.99388,3.16,0.53,9.5,5,0


## LEWIS - Machine Learning Code 

In [102]:
# Clean the dataframe, removing columns that will not be used in the ML model
cleaned_wines_df = wines_df.drop(columns=['id','type', 'quality', 'total_sulfur_dioxide', 'density', 'ph'])

# Display the clean dataframe
cleaned_wines_df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,sulphates,alcohol_wine,quality_categorisation
0,6.2,0.260,0.20,8.00,0.047,35.0,0.42,10.4,1
1,10.2,0.410,0.43,2.20,0.110,11.0,0.67,10.8,0
2,7.1,0.350,0.29,2.50,0.096,20.0,0.65,11.0,1
3,6.9,0.685,0.00,2.50,0.105,22.0,0.57,10.6,1
4,6.7,0.250,0.31,1.35,0.061,30.5,0.53,9.5,0


In [103]:
# Split our preprocessed data into our features and target arrays
X = cleaned_wines_df.drop("quality_categorisation", axis=1)
y = cleaned_wines_df["quality_categorisation"]


In [104]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Fit the StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [105]:
#Define deep neural network model removed third layer
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=24, input_dim=X_train_scaled.shape[1], activation="relu"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [106]:
#Train the model
model.fit(X_train, y_train, epochs=150, batch_size=25)

Epoch 1/150
182/182 [==============================] - 4s 7ms/step - loss: 0.6711 - accuracy: 0.6301
Epoch 2/150
182/182 [==============================] - 1s 6ms/step - loss: 0.6066 - accuracy: 0.6479
Epoch 3/150
182/182 [==============================] - 1s 5ms/step - loss: 0.5981 - accuracy: 0.6501
Epoch 4/150
182/182 [==============================] - 1s 5ms/step - loss: 0.5852 - accuracy: 0.6673
Epoch 5/150
182/182 [==============================] - 1s 6ms/step - loss: 0.5758 - accuracy: 0.6765
Epoch 6/150
182/182 [==============================] - 1s 8ms/step - loss: 0.5721 - accuracy: 0.6800
Epoch 7/150
182/182 [==============================] - 2s 9ms/step - loss: 0.5673 - accuracy: 0.6842
Epoch 8/150
182/182 [==============================] - 2s 10ms/step - loss: 0.5688 - accuracy: 0.6848
Epoch 9/150
182/182 [==============================] - 1s 6ms/step - loss: 0.5587 - accuracy: 0.6974
Epoch 10/150
182/182 [==============================] - 1s 5ms/step - loss: 0.5549 - accur

In [107]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 [==============================] - 0s 3ms/step - loss: 0.5502 - accuracy: 0.7159
Loss: 0.5502374172210693, Accuracy: 0.7158974409103394


## KAI - Correlation Code
##### (please use the [wines_df] DataFrame as it contains the 'quality' column that you will need to calculate the correlations)
